In [57]:
#######################################
# CS 424 - Project 2
# Kevin Kowalski
# Samuel Kajah
# Vijay Vemu
#######################################
#
# This file contains the code to clean
# the original data set. It outputs
# the necessary plots and updated data
# set to be used by shiny.
#
# This file was used for testing and 
# plotting before using RStudio for a
# dashboard.
#
#######################################

In [58]:
# import libraries
library(leaflet)
library(lubridate)
library(dplyr)
library(ggplot2)
library(scales)
library(stringr)
library(hashmap)
library(comprehenr)

In [59]:
# create appropriate column names for data in the following format
"
data rows
col 1 – date YYYYmmDD
col 2 – time HHMM
col 3 – record identifier (factor)
col 4 – storm type (factor)
col 5 – latitude (n/s)
col 6 – longitude (e/w)
col 7 – Maximum sustained wind (in knots)
col 8 – Minimum Pressure (in millibars)
col 9 – 34 kt wind radii maximum extent in northeastern quadrant (in nautical miles) 
col 10 – 34 kt wind radii maximum extent in southeastern quadrant (in nautical miles) 
col 11 – 34 kt wind radii maximum extent in southwestern quadrant (in nautical miles) 
col 12 – 34 kt wind radii maximum extent in northwestern quadrant (in nautical miles) 
col 13 – 50 kt wind radii maximum extent in northeastern quadrant (in nautical miles) 
col 14 – 50 kt wind radii maximum extent in southeastern quadrant (in nautical miles) 
col 15 – 50 kt wind radii maximum extent in southwestern quadrant (in nautical miles) 
col 16 – 50 kt wind radii maximum extent in northwestern quadrant (in nautical miles) 
col 17 – 64 kt wind radii maximum extent in northeastern quadrant (in nautical miles)
col 18 – 64 kt wind radii maximum extent in southeastern quadrant (in nautical miles)
col 19 – 64 kt wind radii maximum extent in southwestern quadrant (in nautical miles) 
col 20 – 64 kt wind radii maximum extent in northwestern quadrant (in nautical miles)
"

data_row_header = c('Date', 'Time', 'Record_ID', 'Storm_Type', 'Lat', 'Lon', 'Speed', 'Pressure', 
                   'Wind_Radii_NE_34', 'Wind_Radii_SE_34', 'Wind_Radii_SW_34', 'Wind_Radii_NW_34',
                   'Wind_Radii_NE_50', 'Wind_Radii_SE_50', 'Wind_Radii_SW_50', 'Wind_Radii_NW_50',
                   'Wind_Radii_NE_64', 'Wind_Radii_SE_64', 'Wind_Radii_SW_64', 'Wind_Radii_NW_64', 'Size')

[1] "\ndata rows\ncol 1 – date YYYYmmDD\ncol 2 – time HHMM\ncol 3 – record identifier (factor)\ncol 4 – storm type (factor)\ncol 5 – latitude (n/s)\ncol 6 – longitude (e/w)\ncol 7 – Maximum sustained wind (in knots)\ncol 8 – Minimum Pressure (in millibars)\ncol 9 – 34 kt wind radii maximum extent in northeastern quadrant (in nautical miles) \ncol 10 – 34 kt wind radii maximum extent in southeastern quadrant (in nautical miles) \ncol 11 – 34 kt wind radii maximum extent in southwestern quadrant (in nautical miles) \ncol 12 – 34 kt wind radii maximum extent in northwestern quadrant (in nautical miles) \ncol 13 – 50 kt wind radii maximum extent in northeastern quadrant (in nautical miles) \ncol 14 – 50 kt wind radii maximum extent in southeastern quadrant (in nautical miles) \ncol 15 – 50 kt wind radii maximum extent in southwestern quadrant (in nautical miles) \ncol 16 – 50 kt wind radii maximum extent in northwestern quadrant (in nautical miles) \ncol 17 – 64 kt wind radii maximum extent in northeastern quadrant (in nautical miles)\ncol 18 – 64 kt wind radii maximum extent in southeastern quadrant (in nautical miles)\ncol 19 – 64 kt wind radii maximum extent in southwestern quadrant (in nautical miles) \ncol 20 – 64 kt wind radii maximum extent in northwestern quadrant (in nautical miles)\n"

In [60]:
# custom functions for reading and formatting data

# determine's if a particular row begins a header for a hurricane entry
is_header_row = function(row) {
    if (sum(is.na(row)) == 15)
        TRUE
    else
        FALSE
}

# create a vector of indices of all hurricane header rows
header_locations = function(data) {
    to_vec(
        for (row in 1: nrow(data))
            if (is_header_row(data[row, ])) 
    row)
}

# create a list of names and data for each hurricane
make_huricane_data_map = function(data, header_indices) {
    names = list()
    name_data = list()
    i = 1 
    for (row_index in 1: length(header_indices)) {
        row = header_indices[row_index]
        current_header = data[row, ]
        names[[i]] = current_header[1, 2]
        if (row_index == length(header_indices)) {
            name_data[[i]] = data[(row + 1): nrow(data), ]
        } else {
            next_header = header_indices[row_index + 1]
            name_data[[i]] = data[(row + 1): (next_header - 1), ]
        }
        i = i + 1
    }
    list(names = names, name_data = name_data)
}

In [61]:
# rename original text files to easier-to-work-with CSV files
file.rename("hurdat2-1851-2018-120319.txt", "hurdat2-1851-2018-120319_atlantic.csv")
file.rename("hurdat2-nepac-1949-2018-122019.txt", "hurdat2-nepac-1949-2018-122019_pacific.csv")

[1] TRUE

[1] TRUE

In [62]:
# read in the renamed CSV files
atlantic_data = read.csv('hurdat2-1851-2018-120319_atlantic.csv', header = FALSE, stringsAsFactors = FALSE)
pacific_data = read.csv('hurdat2-nepac-1949-2018-122019_pacific.csv', header = FALSE, stringsAsFactors = FALSE)

# apply column names to data
colnames(atlantic_data) = data_row_header
colnames(pacific_data) = data_row_header

# get list of all indices with hurricane headers
atlantic_header_indices = header_locations(atlantic_data)
pacific_header_indices = header_locations(pacific_data)

In [63]:
# create a new timestamp column containing both date and time
atlantic_data = atlantic_data %>% mutate(Timestamp = parse_date_time(paste(atlantic_data$Date, atlantic_data$Time, sep = ' '), "Ymd HM", tz = 'America/Chicago', quiet = TRUE))
pacific_data = pacific_data %>% mutate(Timestamp = parse_date_time(paste(pacific_data$Date, pacific_data$Time, sep = ' '), "Ymd HM", tz = 'America/Chicago', quiet = TRUE))

# move new timestamp column to first column
atlantic_data <- atlantic_data[, c(22, 1:21)]
pacific_data <- pacific_data[, c(22, 1:21)]

In [64]:
# write the cleaned data to CSV files
write.csv(atlantic_data, "data_atlantic.csv")
write.csv(pacific_data, "data_pacific.csv")

In [66]:
# print head of data
print("Atlantic Data")
head(atlantic_data)
print("Pacific Data")
head(pacific_data)

[1] "Atlantic Data"


Timestamp,Date,Time,Record_ID,Storm_Type,Lat,Lon,Speed,Pressure,Wind_Radii_NE_34,...,Wind_Radii_NW_34,Wind_Radii_NE_50,Wind_Radii_SE_50,Wind_Radii_SW_50,Wind_Radii_NW_50,Wind_Radii_NE_64,Wind_Radii_SE_64,Wind_Radii_SW_64,Wind_Radii_NW_64,Size
NA,AL011851,UNNAMED,14,,,,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1851-06-25 00:00:00,18510625,0000,,HU,28.0N,94.8W,80,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
1851-06-25 06:00:00,18510625,0600,,HU,28.0N,95.4W,80,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
1851-06-25 12:00:00,18510625,1200,,HU,28.0N,96.0W,80,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
1851-06-25 18:00:00,18510625,1800,,HU,28.1N,96.5W,80,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
1851-06-25 21:00:00,18510625,2100,L,HU,28.2N,96.8W,80,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA


[1] "Pacific Data"


Timestamp,Date,Time,Record_ID,Storm_Type,Lat,Lon,Speed,Pressure,Wind_Radii_NE_34,...,Wind_Radii_NW_34,Wind_Radii_NE_50,Wind_Radii_SE_50,Wind_Radii_SW_50,Wind_Radii_NW_50,Wind_Radii_NE_64,Wind_Radii_SE_64,Wind_Radii_SW_64,Wind_Radii_NW_64,Size
NA,EP011949,UNNAMED,7,,,,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1949-06-11 00:00:00,19490611,0000,,TS,20.2N,106.3W,45,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
1949-06-11 06:00:00,19490611,0600,,TS,20.2N,106.4W,45,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
1949-06-11 12:00:00,19490611,1200,,TS,20.2N,106.7W,45,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
1949-06-11 18:00:00,19490611,1800,,TS,20.3N,107.7W,45,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
1949-06-12 00:00:00,19490612,0000,,TS,20.4N,108.6W,45,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA


In [67]:
# map each individual hurricane for easier access
atlantic_map = make_huricane_data_map(atlantic_data, atlantic_header_indices)
pacific_map = make_huricane_data_map(pacific_data, pacific_header_indices)

In [68]:
# list all hurricanes by cyclone number
print("Atlantic Hurricanes")
atlantic_map$names
print("Pacific Hurricanes")
pacific_map$names

[1] "Atlantic Hurricanes"


[[1]]
[1] "AL011851"

[[2]]
[1] "AL021851"

[[3]]
[1] "AL031851"

[[4]]
[1] "AL041851"

[[5]]
[1] "AL051851"

[[6]]
[1] "AL061851"

[[7]]
[1] "AL011852"

[[8]]
[1] "AL021852"

[[9]]
[1] "AL031852"

[[10]]
[1] "AL041852"

[[11]]
[1] "AL051852"

[[12]]
[1] "AL011853"

[[13]]
[1] "AL021853"

[[14]]
[1] "AL031853"

[[15]]
[1] "AL041853"

[[16]]
[1] "AL051853"

[[17]]
[1] "AL061853"

[[18]]
[1] "AL071853"

[[19]]
[1] "AL081853"

[[20]]
[1] "AL011854"

[[21]]
[1] "AL021854"

[[22]]
[1] "AL031854"

[[23]]
[1] "AL041854"

[[24]]
[1] "AL051854"

[[25]]
[1] "AL011855"

[[26]]
[1] "AL021855"

[[27]]
[1] "AL031855"

[[28]]
[1] "AL041855"

[[29]]
[1] "AL051855"

[[30]]
[1] "AL011856"

[[31]]
[1] "AL021856"

[[32]]
[1] "AL031856"

[[33]]
[1] "AL041856"

[[34]]
[1] "AL051856"

[[35]]
[1] "AL061856"

[[36]]
[1] "AL011857"

[[37]]
[1] "AL021857"

[[38]]
[1] "AL031857"

[[39]]
[1] "AL041857"

[[40]]
[1] "AL011858"

[[41]]
[1] "AL021858"

[[42]]
[1] "AL031858"

[[43]]
[1] "AL041858"

[[44]]
[1] "AL051858"

[[45]]
[1] "AL061858"

[[46]]
[1] "AL011859"

[[47]]
[1] "AL021859"

[[48]]
[1] "AL031859"

[[49]]
[1] "AL041859"

[[50]]
[1] "AL051859"

[[51]]
[1] "AL061859"

[[52]]
[1] "AL071859"

[[53]]
[1] "AL081859"

[[54]]
[1] "AL011860"

[[55]]
[1] "AL021860"

[[56]]
[1] "AL041860"

[[57]]
[1] "AL031860"

[[58]]
[1] "AL051860"

[[59]]
[1] "AL061860"

[[60]]
[1] "AL071860"

[[61]]
[1] "AL011861"

[[62]]
[1] "AL021861"

[[63]]
[1] "AL031861"

[[64]]
[1] "AL041861"

[[65]]
[1] "AL051861"

[[66]]
[1] "AL061861"

[[67]]
[1] "AL071861"

[[68]]
[1] "AL081861"

[[69]]
[1] "AL011862"

[[70]]
[1] "AL021862"

[[71]]
[1] "AL031862"

[[72]]
[1] "AL041862"

[[73]]
[1] "AL051862"

[[74]]
[1] "AL061862"

[[75]]
[1] "AL011863"

[[76]]
[1] "AL021863"

[[77]]
[1] "AL031863"

[[78]]
[1] "AL041863"

[[79]]
[1] "AL051863"

[[80]]
[1] "AL061863"

[[81]]
[1] "AL071863"

[[82]]
[1] "AL081863"

[[83]]
[1] "AL091863"

[[84]]
[1] "AL011864"

[[85]]
[1] "AL021864"

[[86]]
[1] "AL031864"

[[87]]
[1] "AL041864"

[[88]]
[1] "AL051864"

[[89]]
[1] "AL011865"

[[90]]
[1] "AL021865"

[[91]]
[1] "AL031865"

[[92]]
[1] "AL041865"

[[93]]
[1] "AL051865"

[[94]]
[1] "AL061865"

[[95]]
[1] "AL071865"

[[96]]
[1] "AL011866"

[[97]]
[1] "AL021866"

[[98]]
[1] "AL031866"

[[99]]
[1] "AL041866"

[[100]]
[1] "AL051866"

[[101]]
[1] "AL061866"

[[102]]
[1] "AL071866"

[[103]]
[1] "AL011867"

[[104]]
[1] "AL021867"

[[105]]
[1] "AL031867"

[[106]]
[1] "AL041867"

[[107]]
[1] "AL051867"

[[108]]
[1] "AL061867"

[[109]]
[1] "AL071867"

[[110]]
[1] "AL081867"

[[111]]
[1] "AL091867"

[[112]]
[1] "AL011868"

[[113]]
[1] "AL021868"

[[114]]
[1] "AL031868"

[[115]]
[1] "AL041868"

[[116]]
[1] "AL011869"

[[117]]
[1] "AL021869"

[[118]]
[1] "AL031869"

[[119]]
[1] "AL041869"

[[120]]
[1] "AL051869"

[[121]]
[1] "AL061869"

[[122]]
[1] "AL071869"

[[123]]
[1] "AL081869"

[[124]]
[1] "AL091869"

[[125]]
[1] "AL101869"

[[126]]
[1] "AL011870"

[[127]]
[1] "AL021870"

[[128]]
[1] "AL031870"

[[129]]
[1] "AL041870"

[[130]]
[1] "AL051870"

[[131]]
[1] "AL061870"

[[132]]
[1] "AL071870"

[[133]]
[1] "AL081870"

[[134]]
[1] "AL091870"

[[135]]
[1] "AL101870"

[[136]]
[1] "AL111870"

[[137]]
[1] "AL011871"

[[138]]
[1] "AL021871"

[[139]]
[1] "AL031871"

[[140]]
[1] "AL041871"

[[141]]
[1] "AL051871"

[[142]]
[1] "AL061871"

[[143]]
[1] "AL071871"

[[144]]
[1] "AL081871"

[[145]]
[1] "AL011872"

[[146]]
[1] "AL021872"

[[147]]
[1] "AL031872"

[[148]]
[1] "AL041872"

[[149]]
[1] "AL051872"

[[150]]
[1] "AL011873"

[[151]]
[1] "AL021873"

[[152]]
[1] "AL031873"

[[153]]
[1] "AL041873"

[[154]]
[1] "AL051873"

[[155]]
[1] "AL011874"

[[156]]
[1] "AL021874"

[[157]]
[1] "AL031874"

[[158]]
[1] "AL041874"

[[159]]
[1] "AL051874"

[[160]]
[1] "AL061874"

[[161]]
[1] "AL071874"

[[162]]
[1] "AL011875"

[[163]]
[1] "AL021875"

[[164]]
[1] "AL031875"

[[165]]
[1] "AL041875"

[[166]]
[1] "AL051875"

[[167]]
[1] "AL061875"

[[168]]
[1] "AL011876"

[[169]]
[1] "AL021876"

[[170]]
[1] "AL031876"

[[171]]
[1] "AL041876"

[[17

[1] "Pacific Hurricanes"


[[1]]
[1] "EP011949"

[[2]]
[1] "EP021949"

[[3]]
[1] "EP031949"

[[4]]
[1] "EP041949"

[[5]]
[1] "EP051949"

[[6]]
[1] "EP061949"

[[7]]
[1] "EP011950"

[[8]]
[1] "EP021950"

[[9]]
[1] "EP031950"

[[10]]
[1] "CP011950"

[[11]]
[1] "EP041950"

[[12]]
[1] "EP051950"

[[13]]
[1] "EP061950"

[[14]]
[1] "EP011951"

[[15]]
[1] "EP021951"

[[16]]
[1] "EP031951"

[[17]]
[1] "EP041951"

[[18]]
[1] "EP051951"

[[19]]
[1] "EP061951"

[[20]]
[1] "EP071951"

[[21]]
[1] "EP081951"

[[22]]
[1] "EP091951"

[[23]]
[1] "EP011952"

[[24]]
[1] "EP021952"

[[25]]
[1] "EP031952"

[[26]]
[1] "EP041952"

[[27]]
[1] "EP051952"

[[28]]
[1] "EP061952"

[[29]]
[1] "EP071952"

[[30]]
[1] "EP011953"

[[31]]
[1] "EP021953"

[[32]]
[1] "EP031953"

[[33]]
[1] "EP041953"

[[34]]
[1] "EP011954"

[[35]]
[1] "EP021954"

[[36]]
[1] "EP031954"

[[37]]
[1] "EP041954"

[[38]]
[1] "EP051954"

[[39]]
[1] "EP061954"

[[40]]
[1] "EP071954"

[[41]]
[1] "EP081954"

[[42]]
[1] "EP091954"

[[43]]
[1] "EP101954"

[[44]]
[1] "EP111954"

[[45]]
[1] "EP011955"

[[46]]
[1] "EP021955"

[[47]]
[1] "EP031955"

[[48]]
[1] "EP041955"

[[49]]
[1] "EP051955"

[[50]]
[1] "EP061955"

[[51]]
[1] "EP011956"

[[52]]
[1] "EP021956"

[[53]]
[1] "EP031956"

[[54]]
[1] "EP041956"

[[55]]
[1] "EP051956"

[[56]]
[1] "EP061956"

[[57]]
[1] "EP071956"

[[58]]
[1] "EP081956"

[[59]]
[1] "EP091956"

[[60]]
[1] "EP101956"

[[61]]
[1] "EP111956"

[[62]]
[1] "EP011957"

[[63]]
[1] "EP021957"

[[64]]
[1] "EP031957"

[[65]]
[1] "CP011957"

[[66]]
[1] "EP041957"

[[67]]
[1] "EP051957"

[[68]]
[1] "EP061957"

[[69]]
[1] "CP021957"

[[70]]
[1] "EP071957"

[[71]]
[1] "EP081957"

[[72]]
[1] "EP091957"

[[73]]
[1] "EP101957"

[[74]]
[1] "CP031957"

[[75]]
[1] "EP011958"

[[76]]
[1] "EP021958"

[[77]]
[1] "EP031958"

[[78]]
[1] "EP041958"

[[79]]
[1] "EP051958"

[[80]]
[1] "EP061958"

[[81]]
[1] "CP011958"

[[82]]
[1] "EP071958"

[[83]]
[1] "EP081958"

[[84]]
[1] "EP091958"

[[85]]
[1] "EP101958"

[[86]]
[1] "EP111958"

[[87]]
[1] "EP121958"

[[88]]
[1] "EP011959"

[[89]]
[1] "EP021959"

[[90]]
[1] "EP031959"

[[91]]
[1] "EP041959"

[[92]]
[1] "EP051959"

[[93]]
[1] "CP011959"

[[94]]
[1] "EP061959"

[[95]]
[1] "EP071959"

[[96]]
[1] "EP081959"

[[97]]
[1] "EP091959"

[[98]]
[1] "CP021959"

[[99]]
[1] "EP101959"

[[100]]
[1] "CP031959"

[[101]]
[1] "EP111959"

[[102]]
[1] "EP121959"

[[103]]
[1] "EP011960"

[[104]]
[1] "EP021960"

[[105]]
[1] "EP031960"

[[106]]
[1] "EP041960"

[[107]]
[1] "EP051960"

[[108]]
[1] "EP061960"

[[109]]
[1] "EP071960"

[[110]]
[1] "EP011961"

[[111]]
[1] "EP021961"

[[112]]
[1] "EP031961"

[[113]]
[1] "EP041961"

[[114]]
[1] "EP051961"

[[115]]
[1] "EP061961"

[[116]]
[1] "EP071961"

[[117]]
[1] "EP081961"

[[118]]
[1] "EP101961"

[[119]]
[1] "EP011962"

[[120]]
[1] "EP021962"

[[121]]
[1] "EP031962"

[[122]]
[1] "EP041962"

[[123]]
[1] "EP051962"

[[124]]
[1] "EP061962"

[[125]]
[1] "EP071962"

[[126]]
[1] "EP081962"

[[127]]
[1] "EP091962"

[[128]]
[1] "EP011963"

[[129]]
[1] "EP021963"

[[130]]
[1] "EP031963"

[[131]]
[1] "CP011963"

[[132]]
[1] "EP041963"

[[133]]
[1] "EP051963"

[[134]]
[1] "EP061963"

[[135]]
[1] "EP071963"

[[136]]
[1] "EP011964"

[[137]]
[1] "EP021964"

[[138]]
[1] "EP031964"

[[139]]
[1] "EP041964"

[[140]]
[1] "EP051964"

[[141]]
[1] "EP011965"

[[142]]
[1] "EP021965"

[[143]]
[1] "EP031965"

[[144]]
[1] "EP041965"

[[145]]
[1] "EP051965"

[[146]]
[1] "EP061965"

[[147]]
[1] "EP071965"

[[148]]
[1] "EP081965"

[[149]]
[1] "EP091965"

[[150]]
[1] "EP101965"

[[151]]
[1] "EP011966"

[[152]]
[1] "EP021966"

[[153]]
[1] "EP031966"

[[154]]
[1] "EP041966"

[[155]]
[1] "EP051966"

[[156]]
[1] "EP061966"

[[157]]
[1] "EP071966"

[[158]]
[1] "EP081966"

[[159]]
[1] "EP091966"

[[160]]
[1] "EP101966"

[[161]]
[1] "EP111966"

[[162]]
[1] "EP121966"

[[163]]
[1] "EP131966"

[[164]]
[1] "EP011967"

[[165]]
[1] "EP021967"

[[166]]
[1] "EP031967"

[[167]]
[1] "EP041967"

[[168]]
[1] "EP051967"

[[169]]
[1] "EP061967"

[[170]]
[1] "EP071967"

[[171]]
[1] "EP081967"

[[17

In [69]:
# EXAMPLE: get data for first atlantic hurricane
atlantic_map$name_data[1]

,Timestamp,Date,Time,Record_ID,Storm_Type,Lat,Lon,Speed,Pressure,Wind_Radii_NE_34,...,Wind_Radii_NW_34,Wind_Radii_NE_50,Wind_Radii_SE_50,Wind_Radii_SW_50,Wind_Radii_NW_50,Wind_Radii_NE_64,Wind_Radii_SE_64,Wind_Radii_SW_64,Wind_Radii_NW_64,Size
2,1851-06-25 00:00:00,18510625,0000,,HU,28.0N,94.8W,80,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
3,1851-06-25 06:00:00,18510625,0600,,HU,28.0N,95.4W,80,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
4,1851-06-25 12:00:00,18510625,1200,,HU,28.0N,96.0W,80,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
5,1851-06-25 18:00:00,18510625,1800,,HU,28.1N,96.5W,80,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
6,1851-06-25 21:00:00,18510625,2100,L,HU,28.2N,96.8W,80,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
7,1851-06-26 00:00:00,18510626,0000,,HU,28.2N,97.0W,70,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
8,1851-06-26 06:00:00,18510626,0600,,TS,28.3N,97.6W,60,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
9,1851-06-26 12:00:00,18510626,1200,,TS,28.4N,98.3W,60,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
10,1851-06-26 18:00:00,18510626,1800,,TS,28.6N,98.9W,50,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
11,1851-06-27 00:00:00,18510627,0000,,TS,29.0N,99.4W,50,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA


In [70]:
# EXAMPLE: get data for first pacific hurricane
pacific_map$name_data[1]

,Timestamp,Date,Time,Record_ID,Storm_Type,Lat,Lon,Speed,Pressure,Wind_Radii_NE_34,...,Wind_Radii_NW_34,Wind_Radii_NE_50,Wind_Radii_SE_50,Wind_Radii_SW_50,Wind_Radii_NW_50,Wind_Radii_NE_64,Wind_Radii_SE_64,Wind_Radii_SW_64,Wind_Radii_NW_64,Size
2,1949-06-11 00:00:00,19490611,0000,,TS,20.2N,106.3W,45,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
3,1949-06-11 06:00:00,19490611,0600,,TS,20.2N,106.4W,45,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
4,1949-06-11 12:00:00,19490611,1200,,TS,20.2N,106.7W,45,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
5,1949-06-11 18:00:00,19490611,1800,,TS,20.3N,107.7W,45,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
6,1949-06-12 00:00:00,19490612,0000,,TS,20.4N,108.6W,45,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
7,1949-06-12 06:00:00,19490612,0600,,TS,20.5N,109.4W,45,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA
8,1949-06-12 12:00:00,19490612,1200,,TS,20.6N,110.2W,45,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NA


In [71]:
# EXAMPLE: get name of hurricane given cyclone number
str_trim(atlantic_data$Time[which(atlantic_data$Date == "AL092011")])

[1] "IRENE"

In [ ]:
# start plotting, graphing, etc... 
# TODO